In [1]:
import sys
import os
from cobra.io import read_sbml_model

sys.path.append(os.path.abspath("../../src"))

model = read_sbml_model("/Users/azddza/Electromics-project/models/gap_but_models/added_butyrate_exchange_reactions/MAG037_gapfilled_noO2.xml")

print(f"模型中包含 {len(model.reactions)} 个反应，{len(model.metabolites)} 个代谢物，{len(model.genes)} 个基因。")


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
模型中包含 702 个反应，688 个代谢物，344 个基因。


In [2]:
print(model.metabolites.get_by_id("cpd00211_e0").reactions)

frozenset({<Reaction TR_cpd00211_c0 at 0x1739e2860>, <Reaction EX_cpd00211_e0 at 0x1739e26e0>})


In [3]:
for rxn in model.reactions:
    if "cpd00211" in rxn.id:
        print(rxn.id, rxn.reaction)

EX_cpd00211_e0 cpd00211_e0 <=> 
TR_cpd00211_c0 cpd00211_e0 <=> cpd00211_c0


In [4]:
import networkx as nx
G_model = nx.DiGraph()
for reaction in model.reactions:
    reactants = [met.id for met in reaction.reactants]
    products = [met.id for met in reaction.products]
    for r in reactants:
        for p in products:
            G_model.add_edge(r, p, reaction=reaction.id)

In [5]:
print("前10个代谢物节点:")
for i, node in enumerate(G_model.nodes):
    print(node)
    if i > 10: break

print("\n前10条反应边（从代谢物A → B）:")
for i, (u, v, data) in enumerate(G_model.edges(data=True)):
    print(f"{u} → {v} via reaction: {data.get('reaction')}")
    if i > 10: break


前10个代谢物节点:
cpd00002_c0
cpd00012_c0
cpd00018_c0
cpd00067_c0
cpd00279_c0
cpd00010_c0
cpd00142_c0
cpd00031_c0
cpd00001_c0
cpd00295_c0
cpd11420_c0
cpd11421_c0

前10条反应边（从代谢物A → B）:
cpd00002_c0 → cpd00012_c0 via reaction: rxn30387_c0
cpd00002_c0 → cpd00018_c0 via reaction: rxn30387_c0
cpd00002_c0 → cpd00067_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00279_c0 via reaction: rxn00988_c0
cpd00002_c0 → cpd14955_c0 via reaction: rxn07584_c0
cpd00002_c0 → cpd00327_c0 via reaction: rxn09449_c0
cpd00002_c0 → cpd00297_c0 via reaction: rxn01513_c0
cpd00002_c0 → cpd00008_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00009_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00053_c0 via reaction: rxn00187_c0
cpd00002_c0 → cpd15603_c0 via reaction: rxn12848_c0
cpd00002_c0 → cpd11437_c0 via reaction: rxn05249_c0


In [6]:
import networkx as nx

# 读取社区网络图
G = nx.read_graphml("cpd00211_community.graphml")  
print(f"图节点数: {G.number_of_nodes()}, 边数: {G.number_of_edges()}")


图节点数: 20151, 边数: 147336


In [7]:
target_met = "cpd00211_c0"  
if target_met not in G:
    print("目标产物在社区图中不存在")

# 获取与其直接或间接连通的代谢物
from networkx.algorithms.components import connected_components

# 对无向图求子图（假设是无向的）
G_u = G.to_undirected()
components = list(nx.connected_components(G_u))

# 找包含目标物的子图
target_component = next((c for c in components if target_met in c), None)
G_sub = G.subgraph(target_component).copy()


In [8]:
# 从社区图中取出所有代谢物名称（需要确保命名方式一致，如带_c）
community_mets = set(G_sub.nodes)

# 在模型图中找这些代谢物是否能从某些模型已有代谢物可达
exchange_mets = set()
for rxn in model.exchanges:
    for met in rxn.metabolites:
        exchange_mets.add(met.id)

# 在模型图中，从每个 exchange 代谢物出发，检查能否到目标社区
valid_starts = []
for start in exchange_mets:
    for target in community_mets:
        try:
            if nx.has_path(G_model, start, target):
                valid_starts.append(start)
                break
        except:
            continue


In [40]:
print("推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:")
for met in valid_starts:
    print(met)


推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:
cpd00264_e0
cpd00205_e0
cpd00254_e0
cpd00047_e0
cpd00058_e0
cpd00129_e0
cpd10516_e0
cpd15603_e0
cpd00118_e0
cpd00104_e0
cpd00029_e0


In [14]:
import pickle
reaction_dict_path = "reaction_dict.pkl"

# ===== 加载 reaction_dict =====
with open(reaction_dict_path, "rb") as f:
    reaction_dict = pickle.load(f)


In [23]:
from cobra.util.solver import linear_reaction_coefficients
from cobra import Reaction, Metabolite
from cobra.util import solver
from networkx.algorithms.simple_paths import shortest_simple_paths
from cobra.flux_analysis import find_blocked_reactions

#  dead-end 检查函数
def find_dead_ends_manual(model):
    dead_ends = []
    for met in model.metabolites:
        producing = any(rxn.metabolites[met] > 0 for rxn in met.reactions)
        consuming = any(rxn.metabolites[met] < 0 for rxn in met.reactions)
        if not (producing and consuming):
            dead_ends.append(met.id)
    return dead_ends

target_met = "cpd00211_c0"  # 目标代谢物

results = []

for start in valid_starts[:1]:  
    try:
        paths = list(nx.all_simple_paths(G, source=start, target=target_met, cutoff=3))
    except:
        continue

    for path in paths:
        rxn_ids = []
        missing_reactions = []

        # 获取路径中的反应 ID
        for i in range(len(path) - 1):
            try:
                rxn_id = G[path[i]][path[i + 1]]["reaction"]
                rxn_ids.append(rxn_id)
            except:
                continue

        existing_rxns = set(r.id for r in model.reactions)
        to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]

        with model.copy() as m:
            for rxn_id in to_add_rxns:
                if rxn_id in reaction_dict:
                    new_rxn = reaction_dict[rxn_id].copy()
                    m.add_reactions([new_rxn])

            # 添加目标产物的 exchange reaction
            target_ex_id = "EX_cpd00211_e0"
            if target_ex_id not in m.reactions:
                try:
                    met = m.metabolites.get_by_id("cpd00211_e0")
                    target_ex = Reaction(target_ex_id)
                    target_ex.name = "Exchange for cpd00211_e0"
                    target_ex.lower_bound = 0
                    target_ex.upper_bound = 1000
                    target_ex.add_metabolites({met: -1})
                    m.add_reactions([target_ex])
                except KeyError:
                    print(" 目标代谢物 cpd00211_e0 不存在")
                    continue

            m.objective = target_ex_id

            sol = m.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0

            #  添加 blocked reaction 和 dead-end metabolite 检查
            blocked_rxns = set(find_blocked_reactions(m))
            dead_ends = set(find_dead_ends_manual(m))

            rxns_in_path = set(rxn_ids)
            mets_in_path = set()
            for rxn_id in rxn_ids:
                if rxn_id in m.reactions:
                    mets_in_path.update(m.id for m in m.reactions.get_by_id(rxn_id).metabolites)

            blocked_in_path = list(rxns_in_path & blocked_rxns)
            dead_ends_in_path = list(mets_in_path & dead_ends)

            results.append({
                "start": start,
                "path": path,
                "missing_reactions": to_add_rxns,
                "num_missing": len(to_add_rxns),
                "flux": flux,
                "blocked_reactions": blocked_in_path,
                "dead_end_metabolites": dead_ends_in_path
            })


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp806rk3xh.lp
Reading time = 0.09 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphlj272h1.lp
Reading time = 0.00 seconds
: 693 rows, 1411 columns, 6627 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpy1jepr74.lp
Reading time = 0.00 seconds
: 693 rows, 1411 columns, 6627 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpt9fpkrvf.lp
Reading time = 0.00 

Ignoring reaction 'rxn02879' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp1b3kdzbk.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp8f65vxkk.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpclz2mjpa.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

Ignoring reaction 'rxn38354' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp2gnvm3dv.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp9704tmjv.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptivja_ly.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

Ignoring reaction 'rxn45696' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmppqgc6rzl.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpbslj8z7r.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmplhrblnho.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6607 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

In [24]:
results

[{'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00008_c0', 'cpd00001_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn11262', 'rxn48267', 'rxn45696'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn45696', 'rxn48267'],
  'dead_end_metabolites': ['cpd00264_e0',
   'cpd00211_c0',
   'cpd11930_e0',
   'cpd32214_c0',
   'cpd11930_c0',
   'cpd24202_c0']},
 {'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00008_c0', 'cpd00029_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn11262', 'rxn34299', 'rxn13427'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn13427'],
  'dead_end_metabolites': ['cpd00264_e0', 'cpd16910_c0', 'cpd00196_c0']},
 {'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn11262', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd00264_e0', 'cpd01662_c0']},
 {'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cp

In [25]:
all_zero = all(entry['flux'] == 0 for entry in results)
print(f"是否所有 flux 都为 0: {all_zero}")

是否所有 flux 都为 0: True


In [26]:
len(results)

53

In [27]:
# 先按缺失反应数升序排列，缺失越少越靠前
sorted_results = sorted(results, key=lambda r: r['num_missing'])
sorted_results = sorted(sorted_results, key=lambda r: len(r['dead_end_metabolites']), reverse=True)

unique_missing = {}
for item in sorted_results:
    key = tuple(sorted(item['missing_reactions']))
    if key not in unique_missing:
        unique_missing[key] = item
    if len(unique_missing) >= 1000:
        break

sampled_data = list(unique_missing.values())


In [28]:
sampled_data

[{'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00067_c0', 'cpd00001_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn34532', 'rxn48566', 'rxn45696'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn45696', 'rxn48566'],
  'dead_end_metabolites': ['cpd17485_c0',
   'cpd00211_c0',
   'cpd32214_c0',
   'cpd27757_c0',
   'cpd35107_c0',
   'cpd28082_c0',
   'cpd24202_c0']},
 {'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00067_c0', 'cpd00142_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn34532', 'rxn23640', 'rxn00994'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn00994', 'rxn23640'],
  'dead_end_metabolites': ['cpd27896_c0',
   'cpd00120_c0',
   'cpd00211_c0',
   'cpd00279_c0',
   'cpd24320_c0',
   'cpd24314_c0',
   'cpd00142_c0']},
 {'start': 'cpd00264_e0',
  'path': ['cpd00264_e0', 'cpd00008_c0', 'cpd00001_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn11262', 'rxn48267', 'rxn45696'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_react

In [29]:
for item in sampled_data:
#for item in results:
    start = item['start']
    path = item['path']
    to_add_rxns = item['missing_reactions']
    flux = item['flux']
    blocked_in_path = item['blocked_reactions']
    dead_ends_in_path = item['dead_end_metabolites']

    record = {
        "start": start,
        "path": path,
        "missing_reactions": to_add_rxns,
        "num_missing": len(to_add_rxns),
        "flux": flux,
        "blocked_reactions": blocked_in_path,
        "dead_end_metabolites": dead_ends_in_path
    }

    # 尝试补救 dead-end
    extra_rxns = set()
    for rxn_id, rxn in reaction_dict.items():
        for met, coeff in rxn.metabolites.items():
            if met.id in dead_ends_in_path and coeff < 0:  # 消费 dead-end
                extra_rxns.add(rxn_id)
                break

    rescued_flux = 0.0
    if extra_rxns:
        with model.copy() as m2:
            # 添加缺失反应
            for rxn_id in to_add_rxns:
                if rxn_id in reaction_dict:
                    m2.add_reactions([reaction_dict[rxn_id].copy()])
            # 添加补救反应
            for rxn_id in extra_rxns:
                if rxn_id in reaction_dict and rxn_id not in m2.reactions:
                    m2.add_reactions([reaction_dict[rxn_id].copy()])

            try:
                m2.objective = m2.reactions.get_by_id("EX_cpd00211_e0")
                sol2 = m2.optimize()
                rescued_flux = sol2.objective_value if sol2.status == "optimal" else 0.0
            except:
                rescued_flux = 0.0

    record["rescued_reactions"] = list(extra_rxns)
    record["rescued_count"] = len(extra_rxns)
    record["rescued_flux"] = rescued_flux

    results.append(record)


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp_f63rde2.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpxe7m1qmp.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprwbca2re.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpjiry50zc.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptup59lru.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpkitgm47u.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model f

Ignoring reaction 'rxn02879' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptufhguhf.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros


Ignoring reaction 'rxn38354' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpr6f_wewq.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp9_9kxd7y.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros


Ignoring reaction 'rxn45696' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpafu4c01g.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprpe0jcbm.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpscnabh3b.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptrfeq9up.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp2lck76ea.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp_hft8mat.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Read LP format model f

In [32]:
import pandas as pd
df = pd.DataFrame(results)
(df['rescued_flux'] == 1000).any()


False

In [39]:
df["start"].unique

<bound method Series.unique of 0      cpd00264_e0
1      cpd00264_e0
2      cpd00264_e0
3      cpd00264_e0
4      cpd00264_e0
          ...     
99     cpd00264_e0
100    cpd00264_e0
101    cpd00264_e0
102    cpd00264_e0
103    cpd00264_e0
Name: start, Length: 104, dtype: object>

In [33]:
filtered_df = df[df['rescued_flux'] == 1000]
df_f = pd.DataFrame(filtered_df)
df_f

,start,path,missing_reactions,num_missing,flux,blocked_reactions,dead_end_metabolites,rescued_reactions,rescued_count,rescued_flux


In [52]:

def find_minimal_rescue_set(model, reaction_dict, record):

    necessary_rxns = set()
    rescued = record['rescued_reactions']
    missing = record['missing_reactions']

    with model.copy() as base:
        # Step 1: 添加 missing reactions
        for rxn_id in missing:
            if rxn_id in reaction_dict:
                base.add_reactions([reaction_dict[rxn_id].copy()])

        # Step 2: 检查 rescued reactions 哪些是必要的
        for rxn_id in rescued:
            with base.copy() as test_model:
                # 添加除了当前 rxn 之外的 rescued reactions
                for r_id in rescued:
                    if r_id != rxn_id and r_id in reaction_dict:
                        test_model.add_reactions([reaction_dict[r_id].copy()])

                # 添加目标出口反应（如果模型中没有）
                if "EX_cpd00211_e0" not in test_model.reactions:
                    print("Warning: EX_cpd00211_e0 not in model.")
                    continue

                try:
                    test_model.objective = test_model.reactions.get_by_id("EX_cpd00211_e0")
                    sol = test_model.optimize()
                    if sol.status != "optimal" or sol.objective_value < 1e-6:
                        necessary_rxns.add(rxn_id)
                except:
                    necessary_rxns.add(rxn_id)  # 出错就保留
    return list(necessary_rxns)

def update_rescue_info_for_dataframe(df, model, reaction_dict):
    for i, row in df.iterrows():
        print(f"Processing row {i}...")
        try:
            minimal_rxns = find_minimal_rescue_set(model, reaction_dict, row)
            df.at[i, "rescued_reactions"] = minimal_rxns
            df.at[i, "rescued_count"] = len(minimal_rxns)

            with model.copy() as m_final:
                for rxn_id in row['missing_reactions']:
                    if rxn_id in reaction_dict:
                        m_final.add_reactions([reaction_dict[rxn_id].copy()])
                for rxn_id in minimal_rxns:
                    if rxn_id in reaction_dict:
                        m_final.add_reactions([reaction_dict[rxn_id].copy()])
                m_final.objective = m_final.reactions.get_by_id("EX_cpd00211_e0")
                sol_final = m_final.optimize()
                flux = sol_final.objective_value if sol_final.status == "optimal" else 0.0
                df.at[i, "rescued_flux"] = flux
        except Exception as e:
            print(f"Error on row {i}: {e}")
            continue
    return df


In [53]:
updated_df = update_rescue_info_for_dataframe(df_f, model, reaction_dict)
updated_df


Processing row 74...
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpw8yvli0e.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpfviql5_2.lp
Reading time = 0.02 seconds
: 951 rows, 2018 columns, 9060 nonzeros


Ignoring reaction 'rxn13713' since it already exists.
Ignoring reaction 'rxn38807' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpyc3xj6m8.lp
Reading time = 0.01 seconds
: 951 rows, 2018 columns, 9060 nonzeros


Ignoring reaction 'rxn13713' since it already exists.
Ignoring reaction 'rxn38807' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpndq716rd.lp
Reading time = 0.00 seconds
: 951 rows, 2018 columns, 9060 nonzeros


...

Ignoring reaction 'rxn23640' since it already exists.
Ignoring reaction 'rxn00994' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpegcw2vuz.lp
Reading time = 0.01 seconds
: 952 rows, 2018 columns, 9062 nonzeros


Ignoring reaction 'rxn23640' since it already exists.
Ignoring reaction 'rxn00994' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprfs5o4_y.lp
Reading time = 0.00 seconds
: 952 rows, 2018 columns, 9062 nonzeros


KeyboardInterrupt: 

In [74]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("gapfill_results.csv", index=False)



In [92]:
# 假设你已经有 df，其中包含列 rescued_reactions
rescue_rxns = df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0]  # 提取第一行的反应列表
core_path_rxns = ["rxn31359", "rxn13427"]

minimal_set = set(rescue_rxns)
baseline_flux = 1000.0

for rxn in rescue_rxns:
    test_set = minimal_set - {rxn}

    with model.copy() as m:
        for r in core_path_rxns + list(test_set):
            if r in reaction_dict:
                m.add_reactions([reaction_dict[r].copy()])
        
        # 设置目标
        m.objective = m.reactions.get_by_id("EX_cpd00211_e0")
        m.reactions.get_by_id(f"EX_{start}").lower_bound = -10

        flux = m.optimize().objective_value
        if flux >= baseline_flux * 0.999:  # 允许一点浮动
            print(f" 移除 {rxn} 不影响 flux → 删除")
            minimal_set.remove(rxn)
        else:
            print(f" 移除 {rxn} 导致 flux 下降 → 保留")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphgcqle17.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn08184 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpscdr8p9_.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn35683 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp668ifcwi.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


...

 移除 rxn16586 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpsqg11cnx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43573 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprnotss4s.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43619 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpc2de4360.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn13427 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptax9wfxb.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31413 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpu5koggxw.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn03642 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpvqujjgjx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn00994 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0_vlrtls.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31412 不影响 flux → 删除
Read LP format model fr

In [93]:
minimal_set

{'rxn00875', 'rxn10074'}

In [94]:
final_rxns_to_add = core_path_rxns + list(minimal_set)
print(f" 最终要添加的反应总数: {len(final_rxns_to_add)}")


 最终要添加的反应总数: 4


In [96]:
target_rxn_id = "EX_cpd00211_e0"
with model.copy() as m:
    for rxn_id in final_rxns_to_add:
        if rxn_id in reaction_dict:
            m.add_reactions([reaction_dict[rxn_id].copy()])
    m.objective = target_rxn_id
    sol = m.optimize()
    flux = sol.objective_value if sol.status == "optimal" else 0
    print(f"\n 添加最终反应集后的通量: {flux}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp3we0w8rz.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros

 添加最终反应集后的通量: 1000.0


In [ ]:
from cobra.flux_analysis import flux_variability_analysis
# ✅ 主路径反应列表（根据你自己模型设定）
core_path_rxns = ["rxn31359", "rxn13427"]  # 示例，替换为你起点→目标路径的反应

# ✅ 补救反应集（可以从 DataFrame 中提取）
rescue_rxns = list(df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0])

with model.copy() as m:
    for r in core_path_rxns + rescue_rxns:
        if r in reaction_dict:
            m.add_reactions([reaction_dict[r].copy()])

    fva = flux_variability_analysis(m, reaction_list=rescue_rxns)

    essential_rxns = fva[(fva["maximum"].abs() > 1e-6) | (fva["minimum"].abs() > 1e-6)].index.tolist()
    print(f" 最小必需反应集大小: {len(essential_rxns)}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpesrfr49k.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpem3fk5hg.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcalsuv5a.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpgfgfjrvm.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

In [84]:
enhanced_results = []

for res in results:
    start = res["start"]
    base_rxns = res["missing_reactions"]
    dead_mets = res["dead_end_metabolites"]

    for dead_met in dead_mets:
        try:
            # 反向图中从 dead_met → start 找最短路径
            reverse_path = nx.shortest_path(G.reverse(), source=dead_met, target=start)
            print(f"dead-end {dead_met} → 起点 {start} 路径: {reverse_path}")
        except:
            continue

        # 获取修复路径上的反应
        repair_rxns = []
        for i in range(len(reverse_path) - 1):
            try:
                rxn_id = G[reverse_path[i]][reverse_path[i + 1]]["reaction"]
                repair_rxns.append(rxn_id)
            except:
                continue

        # 合并所有需添加的反应
        all_rxns_to_add = list(set(base_rxns + repair_rxns))

        with model.copy() as m:
            for rxn_id in all_rxns_to_add:
                if rxn_id in reaction_dict:
                    new_rxn = reaction_dict[rxn_id].copy()
                    m.add_reactions([new_rxn])

            # 添加起点 exchange
            ex_id = f"EX_{start}"
            if ex_id not in m.reactions:
                try:
                    ex = Reaction(ex_id)
                    ex.lower_bound = -10
                    ex.upper_bound = 1000
                    ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                    m.add_reactions([ex])
                except:
                    continue
            else:
                m.reactions.get_by_id(ex_id).lower_bound = -10

            # 添加目标产物 exchange
            target_ex_id = "EX_cpd00211_e0"
            if target_ex_id not in m.reactions:
                try:
                    target_met = m.metabolites.get_by_id("cpd00211_e0")
                    ex = Reaction(target_ex_id)
                    ex.lower_bound = 0
                    ex.upper_bound = 1000
                    ex.add_metabolites({target_met: -1})
                    m.add_reactions([ex])
                except:
                    continue
            m.objective = target_ex_id

            # 优化
            sol = m.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0
            print(f" 尝试修复 dead-end {dead_met} 后通量: {flux}")

            enhanced_results.append({
                "start": start,
                "original_dead_end": dead_met,
                "original_missing": base_rxns,
                "repair_rxns": repair_rxns,
                "flux_after_repair": flux
            })


dead-end cpd01662_c0 → 起点 cpd00254_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00254_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpm4sc39dk.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00205_e0 → 起点 cpd00205_e0 路径: ['cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp07fyvdax.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd00205_e0 后通量: 0.0
dead-end cpd01662_c0 → 起点 cpd00205_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0zhnosml.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00166_e0 → 起点 cpd00166_e0 路径: ['cpd00166_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0bbz5fg7.lp
Reading time = 0

In [87]:
import pandas as pd

dff = pd.DataFrame(enhanced_results)
dff


,start,original_dead_end,original_missing,repair_rxns,flux_after_repair
0,cpd00254_e0,cpd01662_c0,"[rxn10571, rxn01236]",[rxn00871],0.0
1,cpd00205_e0,cpd00205_e0,"[rxn18910, rxn01236]",[],0.0
2,cpd00205_e0,cpd01662_c0,"[rxn18910, rxn01236]","[rxn00871, rxn13842]",0.0
3,cpd00166_e0,cpd00166_e0,"[rxn18671, rxn01236]",[],0.0
4,cpd00166_e0,cpd01662_c0,"[rxn18671, rxn01236]",[rxn00871],0.0
5,cpd00322_e0,cpd00322_e0,"[rxn08745, rxn01236]",[],0.0
6,cpd00322_e0,cpd01662_c0,"[rxn08745, rxn01236]",[rxn00871],0.0
7,cpd00104_e0,cpd00104_e0,"[rxn05223, rxn01236]",[],0.0
8,cpd00104_e0,cpd01662_c0,"[rxn05223, rxn01236]","[rxn00871, rxn05223]",0.0
9,cpd10515_e0,cpd01662_c0,"[rxn08472, rxn01236]",[rxn00871],0.0
